In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/digit-recognizer/sample_submission.csv
/kaggle/input/digit-recognizer/train.csv
/kaggle/input/digit-recognizer/test.csv


# Import

In [2]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torch.optim.lr_scheduler import ReduceLROnPlateau
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

# Config

In [3]:
batch_size= 64
epochs= 30
learning_rate= 0.001
device= torch.device('cuda' if torch.cuda.is_available() else 'cpu')
seed= 42

torch.manual_seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed(seed)

print(f"using device: {device}")

using device: cuda


In [4]:
train= pd.read_csv('/kaggle/input/digit-recognizer/train.csv')
test= pd.read_csv('/kaggle/input/digit-recognizer/test.csv')
print(train.head())
print(test.head())

   label  pixel0  pixel1  pixel2  pixel3  pixel4  pixel5  pixel6  pixel7  \
0      1       0       0       0       0       0       0       0       0   
1      0       0       0       0       0       0       0       0       0   
2      1       0       0       0       0       0       0       0       0   
3      4       0       0       0       0       0       0       0       0   
4      0       0       0       0       0       0       0       0       0   

   pixel8  ...  pixel774  pixel775  pixel776  pixel777  pixel778  pixel779  \
0       0  ...         0         0         0         0         0         0   
1       0  ...         0         0         0         0         0         0   
2       0  ...         0         0         0         0         0         0   
3       0  ...         0         0         0         0         0         0   
4       0  ...         0         0         0         0         0         0   

   pixel780  pixel781  pixel782  pixel783  
0         0         0         

# Dataset

In [16]:
class Datasetitself(Dataset):
    def __init__(self, data, transform=None, is_test=False):
        self.transform= transform
        self.is_test= is_test

        if self.is_test:
            self.X= data.values.reshape((-1,28,28,1)).astype(np.uint8)
            self.y= None
        else:
            self.X= data.iloc[:, 1:].values.reshape((-1,28,28,1)).astype(np.uint8)
            self.y= torch.from_numpy(data.iloc[:,0].values).type(torch.LongTensor)

    def __len__(self):
        return len(self.X)

    def __getitem__(self,idx):
        images= self.X[idx]

        if self.transform:
            images= self.transform(images)
        if self.is_test:
            return images
        else:
            return images, self.y[idx]

# Data loading

In [18]:
print ('Loading dataset.....')
train_df= pd.read_csv('/kaggle/input/digit-recognizer/train.csv')
test_df= pd.read_csv('/kaggle/input/digit-recognizer/test.csv')
print ('Dataset loaded')

Loading dataset.....
Dataset loaded


# Data processing and augmentation

In [19]:
# train_val split
train_data, val_data = train_test_split(train_df, test_size=0.1, random_state=seed)

train_transform= transforms.Compose([
    transforms.ToPILImage(),
    transforms.RandomRotation(degrees= 10),
    transforms.RandomAffine(degrees= 0, translate=(0.1, 0.1), scale= (0.9,1.1)),
    transforms.ToTensor(),
    transforms.Normalize(mean= [0.5],std=[0.5])
])

val_transform= transforms.Compose([
    transforms.ToPILImage(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])
])

train_dataset= Datasetitself(train_data, transform= train_transform)
val_dataset= Datasetitself(val_data, transform= val_transform)
test_dataset= Datasetitself(test_df, transform= val_transform, is_test=True)

train_loader= DataLoader(train_dataset, batch_size= batch_size, shuffle=True)
val_loader= DataLoader(val_dataset, batch_size= batch_size, shuffle=False)
test_loader= DataLoader(test_dataset, batch_size= batch_size, shuffle=False)

# CNN Model Arch

In [8]:
class MNISTNet(nn.Module):
    def __init__ (self):
        super(MNISTNet, self).__init__()

        #Block 1
        self.conv1= nn.Sequential(
            nn.Conv2d(1,32,kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.Conv2d(32,32,kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Dropout(0.25)
        )

         #Block 2
        self.conv2= nn.Sequential(
            nn.Conv2d(32,64,kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.Conv2d(64,64,kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Dropout(0.25)
        )

        #FC
        self.fc= nn.Sequential(
            nn.Flatten(),
            nn.Linear(64*7*7, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x= self.conv1(x)
        x= self.conv2(x)
        x= self.fc(x)
        return x

model= MNISTNet().to(device)
optimizer = optim.RMSprop(model.parameters(), lr=learning_rate, alpha=0.9)
criterion = nn.CrossEntropyLoss()
scheduler = ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=3)

# Training

In [20]:
print ('Started Training...')

for epoch in range(epochs):
    model.train()
    running_loss= 0.0

    for images, labels in train_loader:
        images, labels= images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs= model(images)
        loss= criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    #Val
    model.eval()
    total= 0
    correct= 0
    with torch.no_grad():
        for images, labels in val_loader:    
            images, labels = images.to(device), labels.to(device)
            outputs= model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    val_acc= correct / total
    scheduler.step(val_acc)

    print(f"Epoch {epoch+1}/{epochs} | Loss: {running_loss/len(train_loader):.4f} | Val Acc: {val_acc:.4f}")
        

Started Training...
Epoch 1/30 | Loss: 0.2349 | Val Acc: 0.9810
Epoch 2/30 | Loss: 0.1095 | Val Acc: 0.9890
Epoch 3/30 | Loss: 0.0915 | Val Acc: 0.9907
Epoch 4/30 | Loss: 0.0818 | Val Acc: 0.9898
Epoch 5/30 | Loss: 0.0737 | Val Acc: 0.9895
Epoch 6/30 | Loss: 0.0791 | Val Acc: 0.9936
Epoch 7/30 | Loss: 0.0754 | Val Acc: 0.9931
Epoch 8/30 | Loss: 0.0694 | Val Acc: 0.9907
Epoch 9/30 | Loss: 0.0690 | Val Acc: 0.9933
Epoch 10/30 | Loss: 0.0663 | Val Acc: 0.9931
Epoch 11/30 | Loss: 0.0568 | Val Acc: 0.9943
Epoch 12/30 | Loss: 0.0522 | Val Acc: 0.9952
Epoch 13/30 | Loss: 0.0496 | Val Acc: 0.9933
Epoch 14/30 | Loss: 0.0488 | Val Acc: 0.9945
Epoch 15/30 | Loss: 0.0507 | Val Acc: 0.9955
Epoch 16/30 | Loss: 0.0482 | Val Acc: 0.9948
Epoch 17/30 | Loss: 0.0497 | Val Acc: 0.9933
Epoch 18/30 | Loss: 0.0497 | Val Acc: 0.9955
Epoch 19/30 | Loss: 0.0517 | Val Acc: 0.9945
Epoch 20/30 | Loss: 0.0476 | Val Acc: 0.9955
Epoch 21/30 | Loss: 0.0452 | Val Acc: 0.9957
Epoch 22/30 | Loss: 0.0452 | Val Acc: 0.9950

# Prediction

In [22]:
print("Generating predictions for test set...")
model.eval()
predictions = []

with torch.no_grad():
    for images in test_loader:
        images = images.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        predictions.extend(predicted.cpu().numpy())

submission = pd.DataFrame({
    "ImageId": range(1, len(predictions) + 1),
    "Label": predictions
})

submission.to_csv("submission.csv", index=False)
print("submission.csv saved successfully!")

Generating predictions for test set...
submission.csv saved successfully!
